In [35]:
import numpy as np
import pandas as pd
#import xarray as xr

import hvplot.pandas  # noqa
import hvplot.xarray  # noqa

import holoviews as hv

In [14]:
deg2rad = np.pi/180

def generate_one_trajectory(u_mean=0, v_mean=0, u_wave=0, noise=0, id=0):

    freq = "1H"

    time_unit = pd.Timedelta("1s")
    dt = pd.Timedelta(freq) / time_unit
    time = pd.date_range(start="2018-01-01", end="2018-01-15", freq=freq)
    _time = (time - time[0]) / time_unit

    lon0, lat0 = -20, 30
    scale_lat = 111e3
    scale_lon = scale_lat * np.cos(lat0 * deg2rad)
    
    T = pd.Timedelta("1D") / time_unit

    u = (
        u_mean
        + u_wave * np.cos(2 * np.pi * _time / T)
        + np.random.randn(time.size) * noise
    )
    v = (
        v_mean
        + u_wave * np.sin(2 * np.pi * _time / T)
        + np.random.randn(time.size) * noise
    )

    lon = lon0 + np.cumsum(u) * dt / scale_lon
    lat = lat0 + np.cumsum(v) * dt / scale_lat

    df = pd.DataFrame(dict(lon=lon, lat=lat, time=time))
    df["id"] = id
    df = df.set_index("time")

    return df

In [19]:
# actually generate one time series
df = generate_one_trajectory(u_mean=-0.1, v_mean=0, u_wave=0.1, noise=0.05, id=0)

In [36]:
dkwargs = dict(hover_cols=["time"], frame_width=400, frame_height=300)

coords = "geo"
coords = dict(x="lon", y="lat", geo=True)

dkwargs = dict(**coords, **dkwargs)
dkwargs["tiles"] = "CartoLight"

hdl = df.hvplot.points(**dkwargs)

In [37]:
hdl

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [lon,lat]   (time)

In [38]:
hv.save(hdl, 'out.html')